In [1]:
# 2021.12.04 / KwnakiAhn / How to setup pyspark on jupyuter

# install java (https://m.blog.naver.com/opusk/220985259485)
# !sudo apt-get install default-jdk

# Install apache spark
# !wget https://spark.apache.org/downloads.html 
# (wget https://archive.apache.org/dist/spark/spark-2.3.0/spark-2.3.0-bin-hadoop2.7.tgz)
# (wget https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz) / EMR release 6.0.0
# !tar -xzf spark-1.2.0-bin-hadoop2.4.tgz
# !sudo mv spark-3.2.0-bin-hadoop3.2 /opt/spark-3.2.0
# !sudo ln -s /opt/spark-3.2.0 /opt/spark̀
# !export SPARK_HOME=/opt/spark
# !export PATH=$SPARK_HOME/bin:$PATH

# install pyspark
# !pip install pyspark
# !pip install findspark

In [2]:
import os, sys
os.environ["SPARK_HOME"] = ['/opt/spark-2.3.0', '/opt/spark-2.4.4', '/opt/spark-3.2.0'][2]
sys.path.insert(0, os.environ["SPARK_HOME"])

import findspark
findspark.init()

# 2021.12.04 / KwankiAhn / How to use graphframes in pyspark
# 1. config('spark.jars.packages') = pyspark --packages graphframes:graphframes:0.6.0-spark2.3-s_2.11
# 2. graphframes <-> spark version sensitive!
#  a. https://spark-packages.org/package/graphframes/graphframes
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("GraphFramePractice1") \
    .config("spark.some.config.option", "some-value") \
    .config('spark.jars.packages', 'graphframes:graphframes:0.8.2-spark3.2-s_2.12')\
    .getOrCreate()

:: loading settings :: url = jar:file:/opt/spark-3.2.0/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jupyter/.ivy2/cache
The jars for the packages stored in: /home/jupyter/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-47849a78-1f6c-4e49-a1c0-a1bba12febdd;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
:: resolution report :: resolve 231ms :: artifacts dl 8ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	-------------------------------

In [3]:
# tutorial guide : https://pizzathief.oopy.io/spark-graphframes-tutorial

# Spark-The-Definitive-Guide 데이터 셋 로드

In [4]:
SPARK_GUIDE_REPO_PATH = "/home/jupyter/src-github/Spark-The-Definitive-Guide"
bikeStations = spark.read.option("header","true")\
  .csv(f"{SPARK_GUIDE_REPO_PATH}/data/bike-data/201508_station_data.csv")
tripData = spark.read.option("header","true")\
  .csv(f"{SPARK_GUIDE_REPO_PATH}/data/bike-data/201508_trip_data.csv")

print(bikeStations.count())
bikeStations.show(4)
print(tripData.count())
tripData.show(4)

70
+----------+--------------------+---------+-----------+---------+--------+------------+
|station_id|                name|      lat|       long|dockcount|landmark|installation|
+----------+--------------------+---------+-----------+---------+--------+------------+
|         2|San Jose Diridon ...|37.329732|-121.901782|       27|San Jose|    8/6/2013|
|         3|San Jose Civic Ce...|37.330698|-121.888979|       15|San Jose|    8/5/2013|
|         4|Santa Clara at Al...|37.333988|-121.894902|       11|San Jose|    8/6/2013|
|         5|    Adobe on Almaden|37.331415|  -121.8932|       19|San Jose|    8/5/2013|
+----------+--------------------+---------+-----------+---------+--------+------------+
only showing top 4 rows



354152
+-------+--------+---------------+--------------------+--------------+---------------+--------------------+------------+------+---------------+--------+
|Trip ID|Duration|     Start Date|       Start Station|Start Terminal|       End Date|         End Station|End Terminal|Bike #|Subscriber Type|Zip Code|
+-------+--------+---------------+--------------------+--------------+---------------+--------------------+------------+------+---------------+--------+
| 913460|     765|8/31/2015 23:26|Harry Bridges Pla...|            50|8/31/2015 23:39|San Francisco Cal...|          70|   288|     Subscriber|    2139|
| 913459|    1036|8/31/2015 23:11|San Antonio Shopp...|            31|8/31/2015 23:28|Mountain View Cit...|          27|    35|     Subscriber|   95032|
| 913455|     307|8/31/2015 23:13|      Post at Kearny|            47|8/31/2015 23:18|   2nd at South Park|          64|   468|     Subscriber|   94107|
| 913454|     409|8/31/2015 23:10|  San Jose City Hall|            10|8/31/

In [5]:
stationVertices = bikeStations.withColumnRenamed("name", "id").distinct()
tripEdges = tripData\
  .withColumnRenamed("Start Station", "src")\
  .withColumnRenamed("End Station", "dst")

stationVertices.show(4)
tripEdges.show(4)

+----------+--------------------+---------+-----------+---------+-------------+------------+
|station_id|                  id|      lat|       long|dockcount|     landmark|installation|
+----------+--------------------+---------+-----------+---------+-------------+------------+
|        51|Embarcadero at Fo...|37.791464|-122.391034|       19|San Francisco|   8/20/2013|
|        58|San Francisco Cit...| 37.77865|-122.418235|       19|San Francisco|   8/21/2013|
|        60|Embarcadero at Sa...| 37.80477|-122.403234|       15|San Francisco|   8/21/2013|
|        65|     Townsend at 7th|37.771058|-122.402717|       15|San Francisco|   8/22/2013|
+----------+--------------------+---------+-----------+---------+-------------+------------+
only showing top 4 rows

+-------+--------+---------------+--------------------+--------------+---------------+--------------------+------------+------+---------------+--------+
|Trip ID|Duration|     Start Date|                 src|Start Terminal|       E

In [6]:
from graphframes import GraphFrame
stationGraph = GraphFrame(stationVertices, tripEdges)
stationGraph.cache()

print('정거장 수: ' + str(stationGraph.vertices.count()) )
print('전체 여행 수: ' + str(stationGraph.edges.count()) )
print('전체 여행 수 원 데이터: ' + str(tripData.count()) )

정거장 수: 70


전체 여행 수: 354152
전체 여행 수 원 데이터: 354152


# 그래프 쿼리하기

In [7]:
import pyspark.sql.functions as F
stationGraph.edges.groupBy('src', 'dst').count().orderBy(F.desc('count')).show(10)

+--------------------+--------------------+-----+
|                 src|                 dst|count|
+--------------------+--------------------+-----+
|San Francisco Cal...|     Townsend at 7th| 3748|
|Harry Bridges Pla...|Embarcadero at Sa...| 3145|
|     2nd at Townsend|Harry Bridges Pla...| 2973|
|     Townsend at 7th|San Francisco Cal...| 2734|
|Harry Bridges Pla...|     2nd at Townsend| 2640|
|Embarcadero at Fo...|San Francisco Cal...| 2439|
|   Steuart at Market|     2nd at Townsend| 2356|
|Embarcadero at Sa...|   Steuart at Market| 2330|
|     Townsend at 7th|San Francisco Cal...| 2192|
|Temporary Transba...|San Francisco Cal...| 2184|
+--------------------+--------------------+-----+
only showing top 10 rows



In [8]:
stationGraph.edges.where("src ='Townsend at 7th' OR dst = 'Townsend at 7th' ").groupBy('src', 'dst').count().show(10)

+--------------------+--------------------+-----+
|                 src|                 dst|count|
+--------------------+--------------------+-----+
|   2nd at South Park|     Townsend at 7th|  312|
|Embarcadero at Br...|     Townsend at 7th|  197|
|     Clay at Battery|     Townsend at 7th|   83|
|       5th at Howard|     Townsend at 7th|  317|
|     Townsend at 7th|South Van Ness at...|  368|
|     Townsend at 7th|San Francisco Cit...|   46|
|   Market at Sansome|     Townsend at 7th|  278|
|     Townsend at 7th|      Market at 10th|  387|
|San Francisco Cal...|     Townsend at 7th| 1198|
|Washington at Kearny|     Townsend at 7th|   60|
+--------------------+--------------------+-----+
only showing top 10 rows



# 서브 그래프
- 규모가 큰 그래프 안에서 형성되는 작은 규모의 그래프를 서브그래프라고 합니다. (그냥 edge 필터해서 새로 그래프 만든거네...)

In [9]:
townAnd7thedges = stationGraph.edges.where("src ='Townsend at 7th' OR dst = 'Townsend at 7th' ")
subgraph = GraphFrame(stationGraph.vertices, townAnd7thedges)

# 모티프 찾기
- 모티프란 구조적 패턴을 그래프로 표현하는 방법입니다. 모티프를 지정하면 실제 데이터 대신 데이터의 패턴을 쿼리합니다. DataFrame에서는 Neo4J의 Cypher 언어와 유사한 도메인에 특화된 언어로 쿼리를 지정하는데, 이 언어를 사용하면 정점과 에지의 조합을 지정하고 그에 대한 이름을 할당할 수 있습니다. 예를 들어 정점 a 가 에지 ab를 통해 다른 정점 b에 연결되도록 하려면 (a)-[ab]→(b) 로 작성합니다. 괄호 또는 대괄호 안의 이름은 값을 나타내는 게 아니라 결과로 나오는 DataFrame에 존재하는 이름과 일치하는 정점 및 가장자리 컬럼 이름입니다, 결과값 반환 목적이 아니면 이름은 생략해도 됩니다. ((a) - [] → () )

In [ ]:
# 1. 세 도착지 간에 삼각형 패턴을 형성하는 모든 자전거를 찾아봅시다. 삼각형 패턴이란 a에서 b로, b에서 c로, c에서 a로 연결되는 에지를 가지는 정점 a,b,c가 있다는 뜻입니다
# 이 코드를 실행하면 정점 a,b,c와 각 에지의 중첩 필드가 포함된 DataFrame 이 생성되고 이제 이 DataFrame에 사용되는 쿼리를 작성할 수 있습니다.
motifs = stationGraph.find('(a)-[ab]->(b); (b)-[bc]->(c); (c)-[ca]->(a)')

# 2. 특정 자전거를 대상으로 위 삼각형 패턴 중 가장 짧은 경로를 찾아봅시다.
# 기존의 타임스탬프를 스파크의 타임스탬프로 파싱한 다음 특정 지점에서 다른 지점으로 이동한 자전거가 동일한 것인지, 각 이동을 시작하는 시점이 올바른지 확인하기 위해 비교를 수행합니다.
# 이때 유의해야 할 점은 모티프 쿼리가 반환한 삼각형(결과값)을 필터링해야 한다는 것입니다. 보통 이러한 쿼리에서는 동일한 정점 ID가 여러 개 있는 경우 하나의 정점 ID로 통합시키지 않으므로, 하나의 정점으로 통합하기 원하면 필터링을 수행해야 합니다.
motifs.selectExpr("*",
    "to_timestamp(ab.`Start Date`, 'MM/dd/yyyy HH:mm') as abStart",
    "to_timestamp(bc.`Start Date`, 'MM/dd/yyyy HH:mm') as bcStart",
    "to_timestamp(ca.`Start Date`, 'MM/dd/yyyy HH:mm') as caStart")\
  .where("ca.`Bike #` = bc.`Bike #`").where("ab.`Bike #` = bc.`Bike #`")\
  .where("a.id != b.id").where("b.id != c.id")\
  .where("abStart < bcStart").where("bcStart < caStart")\
  .orderBy(F.expr("cast(caStart as long) - cast(abStart as long)"))\
  .selectExpr("a.id", "b.id", "c.id", "ab.`Start Date`", "ca.`End Date`")\
  .limit(1).show(1, False)

# 가장 짧은 삼각형 경로는 San Francisco Caltrain 2 → Townsend at 7th → San Francisco Clatrain → San Francisco Caltrain2 순으로 총  22분 정도 걸렸네요.
# 이렇게 모티프 찾기를 결과 테이블 통해 DataFrame 쿼리와 결합하여 발견된 패턴을 더욱 간결히 하거나 정렬, 집계할 수 있다는 것이 GraphFrame의 가장 강력한 기능이라고 합니다.

# 알고리즘

In [11]:
# 1.페이지랭크
ranks = stationGraph.pageRank(resetProbability=0.15, maxIter=10)
ranks.vertices.orderBy(F.desc('pagerank')).select('id', 'pagerank').show(10)

+--------------------+------------------+
|                  id|          pagerank|
+--------------------+------------------+
|San Jose Diridon ...|  4.05150483599002|
|San Francisco Cal...| 3.351183296428697|
|Mountain View Cal...| 2.514390771015544|
|Redwood City Calt...|2.3263087713711714|
|San Francisco Cal...|2.2311442913698833|
|Harry Bridges Pla...|1.8251120118882476|
|     2nd at Townsend|1.5821217785038693|
|Santa Clara at Al...|1.5730074084907586|
|     Townsend at 7th| 1.568456580534055|
|Embarcadero at Sa...| 1.541424208774859|
+--------------------+------------------+
only showing top 10 rows



In [16]:
# 2. In-Degree와 Out-Degree 지표 계산
inDeg = stationGraph.inDegrees
inDeg.orderBy(F.desc('inDegree')).show(5,False)

outDeg = stationGraph.outDegrees
outDeg.orderBy(F.desc('outDegree')).show(5,False)

degreeRatio = inDeg.join(outDeg, 'id').selectExpr('id','double(inDegree)/double(outDegree) as degreeRatio')
degreeRatio.orderBy(F.desc('degreeRatio')).show(10,False)

+----------------------------------------+--------+
|id                                      |inDegree|
+----------------------------------------+--------+
|San Francisco Caltrain (Townsend at 4th)|34810   |
|San Francisco Caltrain 2 (330 Townsend) |22523   |
|Harry Bridges Plaza (Ferry Building)    |17810   |
|2nd at Townsend                         |15463   |
|Townsend at 7th                         |15422   |
+----------------------------------------+--------+
only showing top 5 rows

+---------------------------------------------+---------+
|id                                           |outDegree|
+---------------------------------------------+---------+
|San Francisco Caltrain (Townsend at 4th)     |26304    |
|San Francisco Caltrain 2 (330 Townsend)      |21758    |
|Harry Bridges Plaza (Ferry Building)         |17255    |
|Temporary Transbay Terminal (Howard at Beale)|14436    |
|Embarcadero at Sansome                       |14158    |
+------------------------------------------

In [17]:
# 3. 너비 우선 탐색
stationGraph.bfs(fromExpr="id = 'Townsend at 7th'", toExpr="id = 'Spear at Folsom'", maxPathLength =2).show(10)

+--------------------+--------------------+--------------------+
|                from|                  e0|                  to|
+--------------------+--------------------+--------------------+
|{65, Townsend at ...|{913371, 663, 8/3...|{49, Spear at Fol...|
|{65, Townsend at ...|{913265, 658, 8/3...|{49, Spear at Fol...|
|{65, Townsend at ...|{911919, 722, 8/3...|{49, Spear at Fol...|
|{65, Townsend at ...|{910777, 704, 8/2...|{49, Spear at Fol...|
|{65, Townsend at ...|{908994, 1115, 8/...|{49, Spear at Fol...|
|{65, Townsend at ...|{906912, 892, 8/2...|{49, Spear at Fol...|
|{65, Townsend at ...|{905201, 980, 8/2...|{49, Spear at Fol...|
|{65, Townsend at ...|{904010, 969, 8/2...|{49, Spear at Fol...|
|{65, Townsend at ...|{903375, 850, 8/2...|{49, Spear at Fol...|
|{65, Townsend at ...|{899944, 910, 8/2...|{49, Spear at Fol...|
+--------------------+--------------------+--------------------+
only showing top 10 rows



In [ ]:
# 4. 연결 요소 & 강한 연결 요소
'''
각 Vertex의 Connected Component Membership을 계산하여 각 Vertex에 할당된 Component ID를
갖는 DataFrame을 반환한다. 즉, 연결된 Vertex 끼리는 같은 Component ID를 갖는다.
'''
minGraph = GraphFrame(stationVertices, tripEdges.sample(False, 0.1))
minGraph.connectedComponents().run().show()

In [ ]:
# 5. SCC 찾기
'''
Strongly Connected Components(SCC) 설명
Directed Graph의 어떠한 두 Vertex x와 y에서 x로부터 y까지의 경로와 y로부터 x까지의 경로가 모두 존재하면 x와 y는 Strongly Connected라고 한다. 
즉, Directed Graph에서 x에서 출발하여 y에 도달한 후, 다시 x로 돌아가는 경로가 존재할 경우를 말한다. 
Strongly Connected Components는 이렇게 서로  Strongly Connected된 Vertex들의 집합(또는 Subgraph)를 말한다.

https://m.blog.naver.com/PostView.naver?isHttpsRedirect=true&blogId=8x8x8x8x8x8&logNo=220758152145
'''
scc = minGraph.stronglyConnectedComponents(maxIter=3)
scc.groupBy('component').count().show()